In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [230]:
# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
"""
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
#cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone
"""
# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily.copy()#[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Run Train_Flow

In [7]:
import train_flow
import importlib
importlib.reload(train_flow)

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['include_new']#['exclude_new', 'include_new', 'run_separately']
train_flow.pi_handling_test(pi_handlings)

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""


# ---------- #
# UPDATE ME  #
# ---------- #
new_features = feature_dict['volu']
target_horizons = [2]#, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 1
test_day = 1
feature_cols = feature_dict['ma_lag'] #feature_cols
list_name = "ma_num+exp"


# --------- #
# LEAVE ME  #
# --------- # 
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
results_df, perm_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)

Running for horizon 2 | include_new
86 | 8 | All Cols: ['ADL', 'CMF_20', 'SMA_10_Lag200_min', 'VROC_10', 'VROC_5', 'Vol_Ratio_10_zscore', 'Vol_Ratio_25_zscore', 'Vol_Ratio_50_zscore']
Run 1/1 | Train: 2021-03-24 → 2026-02-06 | Test: 2026-02-11 → 2026-02-11 | Train_n=1225 | Test_n=1 | (PI Years: 1.5 - Feats: 8)


# Run for combo of sets

In [ ]:
import train_flowv2
import importlib
importlib.reload(train_flowv2)
from itertools import combinations

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['run_separately'] #'run_together', 'run_separately'
train_flowv2.pi_handling_test(pi_handlings)
keys = list(feature_dict.keys())
two_combos = list(combinations(keys, 2))

"""
Applicable when appending new features to an existing set
run_separately: runs pi for old and new feature set separately --> combines the top features from each set at end
exclude_new: runs pi only for old feature set then appends all new features onto the trimmed list --> combines the top old list features with all new features 
include_new: runs pi for combined old and new feature set --> if new features aren't adding incremental value they will be completely excluded
"""

master_results = []
master_perm = []

for a, b in two_combos:
# ---------- #
# UPDATE ME  #
# ---------- #
    target_horizons = [2]#, 5]#, 10, 20, 30]
    train_years = [3]
    pi_years = [1.5]
    min_feats = [4]
    days_assessed = 150
    test_day = 1
    feature_cols = feature_dict[a] + feature_dict[b]
    list_name = f"{a}-{b}"
    print(list_name)
    groups = list_name.split("-")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    #models = {"xgboost-3-f": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    models = {"xgboost-1-10": XGBClassifier(n_estimators=100, max_depth=10, colsample_bytree=0.75, min_child_weight=10, random_state=42, n_jobs=-1)}
    models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                        learning_rate=.1, random_state=42, n_jobs=-1)}
    models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    results_df = train_flowv2.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, feature_dict, groups)

    master_results.append(results_df)
    #master_perm.append(perm_df)

# final master DataFrames
#master_results_df = pd.concat(master_results, ignore_index=True)
#master_perm_df    = pd.concat(master_perm, ignore_index=True)

ma_lag-ma_rel
Running for horizon 2 | run_separately
ma_lag: 60 | 6 | ['SMA_10_Lag10_max', 'SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'SMA_10_Lag25_min', 'SMA_10_Lag50_min', 'SMA_25_Lag25_min']
ma_rel: 20 | 10 | ['Close_Rel_Min10', 'Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Close_Rel_Min50', 'Rel_Min_10', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_30', 'Rel_Min_60']
Run 1/150 | Train: 2023-03-06 → 2026-02-06 | Test: 2026-02-11 → 2026-02-11 | Train_n=735 | Test_n=1 | (PI Years: 1.5 - Feats: 4)
Train PPrec: 1.0000 | Train NPrec: 0.9967
ma_lag: 60 | 4 | ['SMA_10_Lag10_max', 'SMA_10_Lag10_min', 'SMA_10_Lag150_min', 'SMA_50_Lag150_min']
ma_rel: 20 | 9 | ['Close_Rel_Min10', 'Close_Rel_Min100', 'Close_Rel_Min200', 'Close_Rel_Min25', 'Close_Rel_Min50', 'Rel_Min_120', 'Rel_Min_240', 'Rel_Min_30', 'Rel_Min_60']
Run 2/150 | Train: 2023-03-03 → 2026-02-05 | Test: 2026-02-10 → 2026-02-10 | Train_n=735 | Test_n=1 | (PI Years: 1.5 - Feats: 4)
Train PPrec: 1.0000 | Train NPrec: 0.9935
ma_la

In [294]:
master_results_df = pd.concat(master_results, ignore_index=True)
base_df = pd.read_csv("h2_performance_all.csv")
cols = base_df.columns
df_concat = pd.concat([base_df.drop_duplicates(), master_results_df[cols].drop_duplicates()], ignore_index=True)
df_concat.to_csv("h2_performance_all.csv", index=False)

# Save most recent run

In [171]:
# ---------- #
# UPDATE ME  #
# ---------- #

for r in target_horizons:
        
        df = pd.read_csv(f"horizon_{r}_baseline_new.csv")


        # --------- #
        # LEAVE ME  #
        # --------- #
        cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
                'pi_size', 'min_feats']
        df_new = master_results_df[cols][master_results_df['horizon'] == r].copy()
        df_concat = pd.concat([df[cols], df_new], ignore_index=True)


        # ---------- #
        # UPDATE ME  #
        # ---------- #
        df_concat.to_csv(f"horizon_{r}_baseline.csv", index=False)

# Run Performance_Flow

In [177]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "horizon_30_baseline.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)

composite_score

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
82,30,xgboost-3,5,ma_sma-experimental_slope,1.5-r,4,0.81,117.0,0.72,0.73,0.20,3.29,0.54,0.99,0.45,0.95,0.98,0.47,0.67
69,30,xgboost-3,5,ma_lag-experimental_slope,1.5-r,4,0.83,116.0,0.71,0.69,0.19,4.13,0.53,0.97,0.45,0.94,0.97,0.42,0.65
63,30,xgboost-3,5,ma_lag-ma_sma,1.5-r,4,0.83,116.0,0.70,0.71,0.18,4.63,0.51,0.96,0.45,0.96,0.96,0.46,0.65
81,30,xgboost-3,5,ma_sma-vix_skew,1.5-r,4,0.82,113.0,0.70,0.72,0.19,1.98,0.50,0.97,0.42,0.89,0.97,0.47,0.64
100,30,xgboost-3,5,ma_sma-pr,1.5-r,4,0.81,116.0,0.69,0.72,0.21,3.34,0.47,0.96,0.42,0.91,0.97,0.46,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,30,xgboost-3,5,ma_num-rsi_macd,1.5-r,4,0.82,120.0,NaN,NaN,0.18,5.01,NaN,0.82,NaN,0.99,0.82,NaN,NaN
77,30,xgboost-3,5,ma_sma-ma_num,1.5-r,4,0.82,120.0,0.58,NaN,0.17,4.17,0.06,0.82,0.33,0.97,0.82,NaN,NaN
101,30,xgboost-3,5,ma_num-pr,1.5-r,4,0.82,119.0,0.41,NaN,0.16,3.66,-0.04,0.81,0.00,0.97,0.83,NaN,NaN
84,30,xgboost-3,5,ma_num-volu,1.5-r,4,0.82,118.0,0.41,NaN,0.17,3.46,-0.04,0.82,0.00,0.97,0.82,NaN,NaN
